In [1]:
import datetime
import pathlib

import parcels
import xarray
from parcels import FieldSet, Field, ParticleSet, JITParticle, AdvectionRK4_3D

from parcels import logger, XarrayDecodedFilter

logger.addFilter(XarrayDecodedFilter())  # Add a filter for the xarray decoding warning

# Subset, lower left corner of my grid
enatl_data_path = pathlib.Path('day_1_5')


In [2]:
mesh_mask_file = enatl_data_path / 'mesh_mask_eNATL60FARSHE_3.6.nc'

u_files = sorted(enatl_data_path.glob('*1d_vozocrtx*'))
v_files = sorted(enatl_data_path.glob('*1d_vomecrty*'))
w_files = sorted(enatl_data_path.glob('*1d_vovecrtz*'))

s_files = sorted(enatl_data_path.glob('*1d_vosaline*'))
t_files = sorted(enatl_data_path.glob('*1d_votemper*'))

lon_lat_w_files = dict(lon=mesh_mask_file, lat=mesh_mask_file, depth=w_files[0])
lon_lat_t_files = dict(lon=mesh_mask_file, lat=mesh_mask_file, depth=s_files[0])

assert len(u_files) == len(w_files)
assert len(u_files) == len(v_files)

In [3]:
variables = {
    'U': 'vozocrtx',
    'V': 'vomecrty',
    'W': 'vovecrtz',
    'S': 'vosaline',
    'T': 'votemper',
}

coords_velocities = dict(lon='glamf', lat='gphif', depth='depthw', time='time_counter')
coords_t_points = dict(lon='glamt', lat='gphit', depth='deptht', time='time_counter')

dimensions = dict(
    U=coords_velocities,
    V=coords_velocities,
    W=coords_velocities,
    S=coords_t_points,
    T=coords_t_points,
)


filenames = dict(
    U=dict(data=u_files, **lon_lat_w_files),
    V=dict(data=v_files, **lon_lat_w_files),
    W=dict(data=w_files, **lon_lat_w_files),
    S=dict(data=s_files, **lon_lat_t_files),
    T=dict(data=t_files, **lon_lat_t_files),
)

fieldset = parcels.FieldSet.from_nemo(
    filenames=filenames,
    variables=variables,
    dimensions=dimensions,
)

In [7]:
def DeleteParticle(particle, fieldset, time):
    particle.delete()


class SampleParticleInitZero(parcels.JITParticle):
    u_vel = parcels.Variable('U', initial=0.0)
    v_vel = parcels.Variable('V', initial=0.0)
    w_vel = parcels.Variable('W', initial=0.0)
    temperature = parcels.Variable('temperature', initial=0.0)
    salinity = parcels.Variable('salinity', initial=0.0)


def sample_field(particle, fieldset, time):
    u, v, w = fieldset.UVW[particle]
    particle.u_vel = u
    particle.v_vel = v
    particle.w_vel = w
    particle.temperature = fieldset.T[time, particle.depth, particle.lat, particle.lon]
    particle.salinity = fieldset.S[time, particle.depth, particle.lat, particle.lon]


def setup_kernel(pset):
    kernel = parcels.AdvectionRK4_3D + pset.Kernel(sample_field)

    return kernel


In [8]:
start_point = (-7.883789, 60.239120)  # y_c = 9, x_r = 21
stop_point = (-7.857898, 60.327419)  # y_c = 18, x_r = 21
depth = 52.48403405  # z_c = 20

pset = ParticleSet.from_line(
    fieldset=fieldset,
    pclass=SampleParticleInitZero,
    size=10,
    start=start_point,
    finish=stop_point,
    depth=depth,
)

In [9]:
kernel = setup_kernel(pset)
# kernel = parcels.AdvectionRK4_3D

AttributeError: Particle type PType<ArraySampleParticleInitZero>::[PVar<lon|<class 'numpy.float64'>>, PVar<lat|<class 'numpy.float64'>>, PVar<depth|<class 'numpy.float64'>>, PVar<time|<class 'numpy.float64'>>, PVar<id|<class 'numpy.int64'>>, PVar<dt|<class 'numpy.float64'>>, PVar<_next_dt|<class 'numpy.float64'>>, PVar<once_written|<class 'numpy.int32'>>, PVar<state|<class 'numpy.int32'>>, PVar<U|<class 'numpy.float32'>>, PVar<V|<class 'numpy.float32'>>, PVar<W|<class 'numpy.float32'>>, PVar<temperature|<class 'numpy.float32'>>, PVar<salinity|<class 'numpy.float32'>>, PVar<ngrids|<class 'numpy.int32'>>, PVar<xi|<class 'numpy.int32'>>, PVar<yi|<class 'numpy.int32'>>, PVar<zi|<class 'numpy.int32'>>, PVar<ti|<class 'numpy.int32'>>] does not define attribute "u_vel".
Please add 'u_vel' to PType<ArraySampleParticleInitZero>::[PVar<lon|<class 'numpy.float64'>>, PVar<lat|<class 'numpy.float64'>>, PVar<depth|<class 'numpy.float64'>>, PVar<time|<class 'numpy.float64'>>, PVar<id|<class 'numpy.in

In [ ]:
output_file = pset.ParticleFile(
    name='output_sampling.zarr', outputdt=datetime.timedelta(hours=6)
)
starttime = datetime.datetime(2010, 1, 1, 12)
endtime = starttime + datetime.timedelta(days=4)
runtime = endtime - starttime

pset.execute(
    kernel,
    runtime=runtime,
    dt=datetime.timedelta(hours=6),
    output_file=output_file,
    recovery={parcels.ErrorCode.ErrorOutOfBounds: DeleteParticle},
)

In [ ]:
parcels.plotTrajectoriesFile('output_sampling.zarr')

In [ ]:
particles = xarray.open_zarr('output_sampling.zarr')

In [ ]:
particles

In [ ]:
particles.S.values